In [1]:
import duckdb
from pathlib import Path
import datetime

In [2]:
ts = datetime.datetime.now()

In [3]:
base_path = Path().cwd().parent
source_path = base_path / Path('raw/entity/anime')
target_path = base_path / Path(f'silver/anilist/tag/tag-{ts.year}-{ts.month}-{ts.day}.parquet')

In [6]:
tb_anime_tag = duckdb.sql(f"""
    WITH source AS(
    SELECT * FROM read_json_auto('{str(source_path)}/*.json')
    )
    , extracted_to_list AS (
    SELECT (source->>'Page'->>'media')::JSON[] as anime_list FROM source
    )
    , unnest_ AS (
    SELECT unnest(anime_list) as anime FROM extracted_to_list
    )
    , extracted_attributes_list AS (
        SELECT
        json_extract(anime,
        [
            'id'
            , '$.tags'

        ]) attribute_list
        FROM unnest_
    )
    , tb_tags AS (
        SELECT
        attribute_list[1]::INT AS id
        , json_extract_string(unnest(attribute_list[2]::JSON[]),
        [
            '$.id',
            '$.name'
        ]) attribute_list_tag
        FROM extracted_attributes_list
    )
    SELECT
        attribute_list_tag[1]::INT AS id
        , attribute_list_tag[2]::VARCHAR AS name
        , id AS anime_id
    FROM tb_tags
""")

In [8]:
tb_anime_tag.to_parquet(str(target_path))